In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
from tensorflow import keras
from tensorflow.keras.layers import LSTM, RepeatVector, GRU, Embedding
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
import numpy as np
import os
from ast import literal_eval as make_tuple
from scipy.sparse import csr_matrix
import pandas as pd
import skmob
from skmob.tessellation import tilers

2023-03-22 15:32:57.369507: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 15:32:58.940224: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-22 15:32:58.941677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-22 15:32:58.943223: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

4 Physical GPUs, 1 Logical GPU


In [2]:
points = pd.read_pickle('../data/freemove/freemove_point_geographical_context.pickle')
points['lat'] = points.geometry.apply(lambda x: x.y)
points['lng'] = points.geometry.apply(lambda x: x.x)

In [3]:
points

,TRIP_ID,PERSON_ID,TIME,geometry,geographical_context,lat,lng
0,978933,17246,2022-10-21 14:59:24,POINT (13.31753 52.53094),"[0.36, 0.22, 0.04, 0.17, 0.46, 0.55, 0.01, 0.3...",52.530942,13.317532
1,978933,17246,2022-10-21 15:02:10,POINT (13.32791 52.53281),"[0.83, 0.35, 0.05, 0.4, 0.99, 0.84, 0.02, 0.27...",52.532806,13.327908
2,978933,17246,2022-10-21 15:10:39,POINT (13.36288 52.53582),"[0.18, 0.27, 0.03, 0.09, 0.49, 0.27, 0.16, 0.2...",52.535821,13.362884
3,978933,17246,2022-10-21 15:13:28,POINT (13.36931 52.52905),"[0.27, 0.4, 0.07, 0.16, 0.45, 0.53, 0.14, 0.19...",52.529052,13.369314
4,978933,17246,2022-10-21 15:16:03,POINT (13.36997 52.52837),"[0.26, 0.39, 0.08, 0.15, 0.43, 0.55, 0.25, 0.2...",52.528374,13.369971
...,...,...,...,...,...,...,...
12346,1015191,16370,2022-11-17 22:09:33,POINT (13.36541 52.47671),"[0.09, 0.26, 0.01, 0.03, 0.28, 0.34, 0.02, 0.0...",52.476712,13.365414
12347,1015191,16370,2022-11-17 22:15:28,POINT (13.35614 52.46070),"[0.02, 0.48, 0.0, 0.02, 1.0, 0.34, 0.15, 0.02,...",52.460702,13.356144
12348,1015191,16370,2022-11-17 22:17:07,POINT (13.35593 52.45973),"[0.02, 0.59, 0.0, 0.03, 1.0, 0.34, 0.22, 0.02,...",52.459734,13.355926
12349,1015191,16370,2022-11-17 22:19:27,POINT (13.35338 52.44808),"[0.29, 0.39, 0.0, 0.07, 0.3, 0.22, 0.16, 0.05,...",52.448084,13.353378


## Filter trajectories that lie outside of berlin


In [6]:
tessellation = tilers.tiler.get("squared", base_shape='Berlin, Germany', meters=500)

tdf = skmob.TrajDataFrame(points)
mapped = tdf.mapping(tessellation, remove_na=True)

/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3460: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/skmob/core/trajectorydataframe.py:322: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  tile_ids = gpd.sjoin(gdf, tes

In [7]:
filtered_indices = set(points.index).difference(mapped.index)

In [8]:
drop_traj_ids = []
for i, point in points.iterrows():
    if i in filtered_indices:
        drop_traj_ids.append(point.TRIP_ID)

In [9]:
len(drop_traj_ids) == len(points) - len(mapped)

True

In [10]:
points = points.query('TRIP_ID not in @drop_traj_ids').reset_index(drop=True).copy()

## Filter users with less than n trips


In [11]:
n = 1

# Count the number of unique values for each ID
unique_counts = points.groupby('PERSON_ID')['TRIP_ID'].nunique()
# Filter out the IDs with less than n unique values
filtered_ids = unique_counts[unique_counts >= n].index.tolist()
points = points[points['PERSON_ID'].isin(filtered_ids)].reset_index(drop=True)

## Geo Context

In [12]:
max_points = points.groupby('TRIP_ID').count()['PERSON_ID'].max()
geographical_context_dim = len(points['geographical_context'].iloc[0])

X_geo_context = np.zeros((points.TRIP_ID.nunique(), max_points, geographical_context_dim))
Y = np.zeros((points.TRIP_ID.nunique(),))

In [13]:
# convert points into numpy array for each trajectory
for index, traj in enumerate(points.groupby('TRIP_ID')):    
    # get the trajectory id
    traj_id = traj[0]

    # get the user id
    user_id = traj[1]['PERSON_ID'].iloc[0]

    Y[index] = user_id

    for idx, point in traj[1].reset_index(drop=True).iterrows():
        X_geo_context[index, idx] = point['geographical_context']

In [14]:
print(X_geo_context.shape, X_geo_context[0])

(1294, 36, 10) [[0.36 0.22 0.04 0.17 0.46 0.55 0.01 0.3  1.   0.02]
 [0.83 0.35 0.05 0.4  0.99 0.84 0.02 0.27 1.   0.01]
 [0.18 0.27 0.03 0.09 0.49 0.27 0.16 0.21 1.   0.01]
 [0.27 0.4  0.07 0.16 0.45 0.53 0.14 0.19 1.   0.  ]
 [0.26 0.39 0.08 0.15 0.43 0.55 0.25 0.2  1.   0.  ]
 [0.18 0.27 0.02 0.09 0.51 0.25 0.2  0.2  1.   0.01]
 [0.46 0.29 0.04 0.15 0.75 0.47 0.1  0.25 1.   0.03]
 [0.44 0.32 0.06 0.12 0.76 0.45 0.16 0.3  1.   0.04]
 [0.44 0.32 0.06 0.12 0.77 0.45 0.16 0.3  1.   0.05]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.  

## Geo type

In [15]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
geo_type = np.asarray(points.geographical_context.apply(lambda x: np.argmax(x)).tolist()).reshape(-1,1)

In [17]:
enc_geo_type = OneHotEncoder(handle_unknown='ignore')

enc_geo_type.fit(geo_type)
enc_geo_type.categories_

[array([0, 1, 4, 5, 6, 7, 8])]

In [18]:
points['geo_type'] = enc_geo_type.transform(geo_type).toarray().tolist()

In [19]:
geo_type_dim = len(points['geo_type'].iloc[0])


X_geo_type = np.zeros((points.TRIP_ID.nunique(), max_points, geo_type_dim))

# convert points into numpy array for each trajectory
for index, traj in enumerate(points.groupby('TRIP_ID')):    
    for idx, point in traj[1].reset_index(drop=True).iterrows():
        X_geo_type[index, idx, :] = point['geo_type']

In [20]:
print(X_geo_type.shape, X_geo_type[0])

(1294, 36, 7) [[0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


## Geohash

In [21]:
import geohash

In [22]:
points['bin_geohash'] = points.geometry.apply(lambda x: geohash.bin_geohash(x.y, x.x, precision=8))

In [23]:
geohash_dim = len(points['bin_geohash'].iloc[0])


X_geohash = np.zeros((points.TRIP_ID.nunique(), max_points, geohash_dim))

# convert points into numpy array for each trajectory
for index, traj in enumerate(points.groupby('TRIP_ID')):    
    for idx, point in traj[1].reset_index(drop=True).iterrows():
        X_geohash[index, idx, :] = point['bin_geohash']

In [24]:
print(X_geohash.shape, X_geohash[0])

(1294, 36, 40) [[1. 1. 0. ... 1. 0. 0.]
 [1. 1. 0. ... 0. 0. 1.]
 [1. 1. 0. ... 1. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Time

In [25]:
enc_time = OneHotEncoder(handle_unknown='ignore')

In [26]:
# get hour from TIME column and reshape to array wiht one feature
hour = np.asarray(points.TIME.dt.hour).reshape(-1,1)

In [27]:
enc_time.fit(hour)
enc_time.categories_

[array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23])]

In [28]:
points['hour'] = enc_time.transform(hour).toarray().tolist()

In [29]:
hour_dim = len(points['hour'].iloc[0])


X_hour = np.zeros((points.TRIP_ID.nunique(), max_points, hour_dim))

# convert points into numpy array for each trajectory
for index, traj in enumerate(points.groupby('TRIP_ID')):    
    for idx, point in traj[1].reset_index(drop=True).iterrows():
        X_hour[index, idx, :] = point['hour']

In [30]:
print(X_hour.shape, X_hour[0])

(1294, 36, 23) [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

## Merge Inputs

In [31]:
from tensorflow.keras.layers import Concatenate

keys = ['bin_geohash', 'hour', 'geographical_context', 'geo_type']

X = [X_geohash, X_hour, X_geo_context, X_geo_type]

vocab_size = []
X = Concatenate(axis=2)(X)


## Autoencoder

In [32]:
from tensorflow.keras.layers import LSTM, RepeatVector, GRU, Embedding, Dense, TimeDistributed, Lambda, Bidirectional, Masking
from keras.initializers import he_uniform
from keras.regularizers import l1
from attention import Attention


In [86]:
latent_dim = 500
EMBEDDER_SIZE = 150
timesteps = int(max_points)

# input_geohash = keras.Input(shape=(timesteps,geohash_dim))
# input_hour = keras.Input(shape=(timesteps,hour_dim))
# input_geo_context = keras.Input(shape=(timesteps,geographical_context_dim))
# input_geo_type = keras.Input(shape=(timesteps,geo_type_dim))

# inputs = [input_geohash, input_hour, input_geo_context, input_geo_type]
# hidden_input = Concatenate(axis=2)(inputs)

inputs = keras.Input(shape=(timesteps,geohash_dim+hour_dim+geographical_context_dim+geo_type_dim))

# masked = Masking(mask_value=0.,
#                 input_shape=(timesteps, features))(inputs)

# e_geohash = Embedding(geohash_dim,
#                       EMBEDDER_SIZE,
#                       input_length=timesteps,
#                       name='e_geohash')(input_geohash)
# e_hour = Embedding(hour_dim,
#                       EMBEDDER_SIZE,
#                       input_length=timesteps,
#                       name='e_hour')(input_hour)
# e_geo_context = Embedding(geographical_context_dim,
#                       EMBEDDER_SIZE,
#                       input_length=timesteps,
#                       name='e_geo_context')(input_geo_context)
# e_geo_type = Embedding(geo_type_dim,
#                       EMBEDDER_SIZE,
#                       input_length=timesteps,
#                       name='e_geo_type')(input_geo_type)

# e_geohash = Dense(units=EMBEDDER_SIZE, activation='relu', use_bias=True, kernel_initializer=he_uniform(seed=1))(input_geohash)
# e_hour = Dense(units=EMBEDDER_SIZE, activation='relu', use_bias=True, kernel_initializer=he_uniform(seed=1))(input_hour)
# e_geo_context = Dense(units=EMBEDDER_SIZE, activation='relu', use_bias=True, kernel_initializer=he_uniform(seed=1))(input_geo_context)
# e_geo_type = Dense(units=EMBEDDER_SIZE, activation='relu', use_bias=True, kernel_initializer=he_uniform(seed=1))(input_geo_type)

# embeddings = [e_geohash, e_hour, e_geo_context, e_geo_type]
# hidden_input = Concatenate(axis=2)(embeddings)

encoded = Dense(units=128, activation='relu', use_bias=True, kernel_initializer=he_uniform(seed=1))(inputs)
encoded = Dense(units=256, activation='relu', use_bias=True, kernel_initializer=he_uniform(seed=1))(encoded)

encoded = Bidirectional(GRU(latent_dim, return_sequences=True, recurrent_regularizer=l1(0.02)))(encoded)
encoded = Bidirectional(GRU(latent_dim, return_sequences=False, recurrent_regularizer=l1(0.02)))(encoded)

# encoded = Attention(units=500)(encoded)

z_mean = Dense(latent_dim)(encoded)
z_log_sigma = Dense(latent_dim)(encoded)

decoded = RepeatVector(timesteps)(encoded)
decoded = Bidirectional(GRU(latent_dim, return_sequences=True, recurrent_regularizer=l1(0.02)))(decoded)
decoded = Bidirectional(GRU(latent_dim, return_sequences=True, recurrent_regularizer=l1(0.02)))(decoded)

decoded = Dense(units=256, activation='relu', use_bias=True, kernel_initializer=he_uniform(seed=1))(decoded)
decoded = Dense(units=128, activation='relu', use_bias=True, kernel_initializer=he_uniform(seed=1))(decoded)

output_geohash = TimeDistributed(Dense(geohash_dim, kernel_initializer=he_uniform(), activation='sigmoid'), name='output_geohash')(decoded)
output_hour = TimeDistributed(Dense(hour_dim, kernel_initializer=he_uniform(), activation='softmax'), name='output_hour')(decoded)
output_geo_context = TimeDistributed(Dense(geographical_context_dim, kernel_initializer=he_uniform(), activation='tanh'), name='output_geo_context')(decoded)
output_geo_type = TimeDistributed(Dense(geo_type_dim, kernel_initializer=he_uniform(), activation='softmax'), name='output_geo_type')(decoded)
outputs = [output_geohash, output_hour, output_geo_context, output_geo_type]

outputs = Concatenate(axis=2)(outputs)

# d_4 = Dense(units=2000, activation='relu', use_bias=True, kernel_initializer=he_uniform(seed=1))(decoded)
# d_5 = Dense(units=500, activation='relu', use_bias=True, kernel_initializer=he_uniform(seed=1))(d_4)
# d_6 = Dense(units=500, activation='relu', use_bias=True, kernel_initializer=he_uniform(seed=1))(d_5)

sequence_autoencoder = keras.Model(inputs, outputs)
encoder = keras.Model(inputs, encoded)
decoder = keras.Model(encoded, outputs)

opt = Adam(learning_rate=0.001)


# sequence_autoencoder.compile(optimizer='adam', 
#                              metrics=['accuracy'],
#                             loss={'output_geohash': 'binary_crossentropy', 'output_hour': 'categorical_crossentropy', 'output_geo_context': 'mse', 'output_geo_type': 'categorical_crossentropy'})

sequence_autoencoder.compile(optimizer='adam', 
                             metrics=['accuracy'],
                            loss='mse')

In [87]:
sequence_autoencoder.summary()

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 36, 80)]     0           []                               
                                                                                                  
 dense_30 (Dense)               (None, 36, 128)      10368       ['input_7[0][0]']                
                                                                                                  
 dense_31 (Dense)               (None, 36, 256)      33024       ['dense_30[0][0]']               
                                                                                                  
 bidirectional_12 (Bidirectiona  (None, 36, 1000)    2274000     ['dense_31[0][0]']               
 l)                                                                                        

In [63]:
sequence_autoencoder.fit(X, X,
                epochs=500,
                batch_size=256,
                shuffle=True)

Epoch 1/500
6/6 [==============================] - 10s 55ms/step - loss: 2244.1433 - accuracy: 0.0000e+00
Epoch 2/500
6/6 [==============================] - 0s 44ms/step - loss: 1654.4951 - accuracy: 0.0000e+00
Epoch 3/500
6/6 [==============================] - 0s 44ms/step - loss: 1197.2808 - accuracy: 0.0000e+00
Epoch 4/500
6/6 [==============================] - 0s 43ms/step - loss: 850.8095 - accuracy: 1.5027e-04
Epoch 5/500
6/6 [==============================] - 0s 43ms/step - loss: 593.3475 - accuracy: 0.0000e+00
Epoch 6/500
6/6 [==============================] - 0s 45ms/step - loss: 407.3020 - accuracy: 0.0027
Epoch 7/500
6/6 [==============================] - 0s 44ms/step - loss: 275.9765 - accuracy: 0.0051
Epoch 8/500
6/6 [==============================] - 0s 44ms/step - loss: 185.5991 - accuracy: 0.0000e+00
Epoch 9/500
6/6 [==============================] - 0s 44ms/step - loss: 123.7784 - accuracy: 2.1467e-05
Epoch 10/500
6/6 [==============================] - 0s 44ms/step - l

6/6 [==============================] - 0s 44ms/step - loss: 14.8668 - accuracy: 0.0026
Epoch 82/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9398 - accuracy: 0.0019
Epoch 83/500
6/6 [==============================] - 0s 44ms/step - loss: 15.0836 - accuracy: 0.0017
Epoch 84/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9970 - accuracy: 0.0027
Epoch 85/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9309 - accuracy: 0.0046
Epoch 86/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9936 - accuracy: 0.0028
Epoch 87/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9597 - accuracy: 0.0028
Epoch 88/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9938 - accuracy: 0.0031
Epoch 89/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9040 - accuracy: 0.0031
Epoch 90/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9080 - accuracy: 0.0039
Epoch 91/500


6/6 [==============================] - 0s 45ms/step - loss: 15.0297 - accuracy: 0.0102
Epoch 163/500
6/6 [==============================] - 0s 45ms/step - loss: 15.0390 - accuracy: 0.0079
Epoch 164/500
6/6 [==============================] - 0s 45ms/step - loss: 14.8965 - accuracy: 0.0111
Epoch 165/500
6/6 [==============================] - 0s 45ms/step - loss: 14.7887 - accuracy: 0.0076
Epoch 166/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9181 - accuracy: 0.0065
Epoch 167/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9249 - accuracy: 0.0079
Epoch 168/500
6/6 [==============================] - 0s 44ms/step - loss: 15.0028 - accuracy: 0.0056
Epoch 169/500
6/6 [==============================] - 0s 44ms/step - loss: 15.0040 - accuracy: 0.0052
Epoch 170/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9410 - accuracy: 0.0079
Epoch 171/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9312 - accuracy: 0.0100
Epoc

6/6 [==============================] - 0s 45ms/step - loss: 14.9244 - accuracy: 0.0993
Epoch 244/500
6/6 [==============================] - 0s 45ms/step - loss: 14.8743 - accuracy: 0.1047
Epoch 245/500
6/6 [==============================] - 0s 44ms/step - loss: 14.8569 - accuracy: 0.1103
Epoch 246/500
6/6 [==============================] - 0s 45ms/step - loss: 15.0170 - accuracy: 0.1166
Epoch 247/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9573 - accuracy: 0.1222
Epoch 248/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9329 - accuracy: 0.1260
Epoch 249/500
6/6 [==============================] - 0s 44ms/step - loss: 14.8480 - accuracy: 0.1253
Epoch 250/500
6/6 [==============================] - 0s 45ms/step - loss: 14.8365 - accuracy: 0.1274
Epoch 251/500
6/6 [==============================] - 0s 45ms/step - loss: 14.8893 - accuracy: 0.1294
Epoch 252/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9540 - accuracy: 0.1338
Epoc

6/6 [==============================] - 0s 44ms/step - loss: 14.9101 - accuracy: 0.1590
Epoch 325/500
6/6 [==============================] - 0s 46ms/step - loss: 14.8034 - accuracy: 0.1607
Epoch 326/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9186 - accuracy: 0.1620
Epoch 327/500
6/6 [==============================] - 0s 45ms/step - loss: 14.8346 - accuracy: 0.1613
Epoch 328/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9004 - accuracy: 0.1624
Epoch 329/500
6/6 [==============================] - 0s 44ms/step - loss: 14.8887 - accuracy: 0.1615
Epoch 330/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9489 - accuracy: 0.1623
Epoch 331/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9195 - accuracy: 0.1617
Epoch 332/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9108 - accuracy: 0.1632
Epoch 333/500
6/6 [==============================] - 0s 45ms/step - loss: 14.8541 - accuracy: 0.1631
Epoc

6/6 [==============================] - 0s 45ms/step - loss: 14.7221 - accuracy: 0.1595
Epoch 406/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9227 - accuracy: 0.1623
Epoch 407/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9694 - accuracy: 0.1624
Epoch 408/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9210 - accuracy: 0.1613
Epoch 409/500
6/6 [==============================] - 0s 44ms/step - loss: 14.8306 - accuracy: 0.1615
Epoch 410/500
6/6 [==============================] - 0s 44ms/step - loss: 14.8239 - accuracy: 0.1613
Epoch 411/500
6/6 [==============================] - 0s 45ms/step - loss: 14.8505 - accuracy: 0.1611
Epoch 412/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9289 - accuracy: 0.1604
Epoch 413/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9354 - accuracy: 0.1580
Epoch 414/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9032 - accuracy: 0.1581
Epoc

6/6 [==============================] - 0s 44ms/step - loss: 14.7889 - accuracy: 0.1647
Epoch 486/500
6/6 [==============================] - 0s 44ms/step - loss: 14.8842 - accuracy: 0.1673
Epoch 487/500
6/6 [==============================] - 0s 45ms/step - loss: 14.8692 - accuracy: 0.1672
Epoch 488/500
6/6 [==============================] - 0s 44ms/step - loss: 14.8756 - accuracy: 0.1688
Epoch 489/500
6/6 [==============================] - 0s 44ms/step - loss: 14.7521 - accuracy: 0.1696
Epoch 490/500
6/6 [==============================] - 0s 45ms/step - loss: 14.8421 - accuracy: 0.1707
Epoch 491/500
6/6 [==============================] - 0s 45ms/step - loss: 14.9889 - accuracy: 0.1718
Epoch 492/500
6/6 [==============================] - 0s 44ms/step - loss: 14.9320 - accuracy: 0.1728
Epoch 493/500
6/6 [==============================] - 0s 44ms/step - loss: 14.8735 - accuracy: 0.1726
Epoch 494/500
6/6 [==============================] - 0s 46ms/step - loss: 14.7841 - accuracy: 0.1738
Epoc

## Evaluation & Simple Clustering

In [88]:
import sys
sys.path.append('..')
import attack
import geopandas as gp
gdf = gp.read_file('../data/freemove/raw_full.geojson', crs='EPSG:4326')
gdf = gdf[gdf['TRIP_ID'].isin(points.TRIP_ID)]
true_mapping = gdf[['TRIP_ID', 'PERSON_ID']].sort_values('TRIP_ID').drop_duplicates().reset_index(drop=True)

print('Number of persons in data', points.PERSON_ID.nunique())

Number of persons in data 72


In [89]:
# Evaluate base line hdbscan
y = attack.getGroundTruth(true_mapping)
import hdbscan
from sklearn import metrics
from sklearn.cluster import KMeans, DBSCAN

pred = encoder.predict(X)

h_dbscan = hdbscan.HDBSCAN()
h_dbscan.fit(pred)

kmeans = KMeans(n_clusters=72, random_state=0, n_init=20).fit(pred)

dbscan = DBSCAN(eps=0.5, min_samples=10).fit(pred)

attack.evaluate(h_dbscan.labels_, true_mapping)
attack.evaluate(kmeans.labels_, true_mapping)
attack.evaluate(dbscan.labels_, true_mapping)

41/41 [==============================] - 1s 10ms/step
Homogeneity: 0.022
Completeness: 0.234
V-measure: 0.040
Rand index: 0.207
ARI: 0.001
MI: 0.085
NMI: 0.040
AMI: 0.021
Cluster accuracy: 0.068
Homogeneity: 0.536
Completeness: 0.497
V-measure: 0.516
Rand index: 0.967
ARI: 0.142
MI: 2.090
NMI: 0.516
AMI: 0.306
Cluster accuracy: 0.278
Homogeneity: 0.000
Completeness: 1.000
V-measure: 0.000
Rand index: 0.024
ARI: 0.000
MI: 0.000
NMI: 0.000
AMI: 0.000
Cluster accuracy: 0.058


## DETECT Joint Optimization

In [67]:
from keras.layers import Dense, LSTM, RepeatVector, GRU, Bidirectional, InputSpec
import keras.backend as K


class ClusteringLayer(keras.layers.Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
         Measure the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [68]:
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [69]:
loss = 0
index = 0
maxiter = 8000
update_interval = 140
index_array = np.arange(X.shape[0])

batch_size = 256

tol = 0.001 # tolerance threshold to stop training

# Simple (First train AE then refine Clustering with KLD loss)

In [83]:
# Initialize Clustering Layer KMEANS
from sklearn.cluster import KMeans, DBSCAN

N_CLUSTERS = 72

# Build clustering model
clustering_layer = ClusteringLayer(N_CLUSTERS, name='clustering')(encoder.output)
model = keras.Model(inputs=encoder.input, outputs=clustering_layer)
model.compile(optimizer=opt, loss='kld')

# Initialize cluster centers using k-means.
kmeans = KMeans(n_clusters=N_CLUSTERS, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(X))
y_pred_last = np.copy(y_pred)
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

# Get labels (ground truth)
y = attack.getGroundTruth(true_mapping)



41/41 [==============================] - 1s 9ms/step


In [71]:
from sklearn import metrics

for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q = model.predict(X, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y is not None:
            acc = np.round(attack.cluster_acc(y, y_pred), 5)
            nmi = np.round(metrics.normalized_mutual_info_score(y, y_pred), 5)
            ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f' % (ite, acc, nmi, ari), ' ; loss=', loss)

        # check stop criterion - model convergence
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, X.shape[0])]
    loss = model.train_on_batch(x=tf.gather(X, indices=idx), y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= X.shape[0] else 0

Iter 0: acc = 0.28825, nmi = 0.53513, ari = 0.15959  ; loss= 0
Iter 140: acc = 0.29289, nmi = 0.53364, ari = 0.15715  ; loss= 4.98642
Iter 280: acc = 0.29598, nmi = 0.53360, ari = 0.15579  ; loss= 5.7236
Iter 420: acc = 0.28516, nmi = 0.52341, ari = 0.14236  ; loss= 6.825
Iter 560: acc = 0.28748, nmi = 0.52328, ari = 0.14179  ; loss= 7.2276
Iter 700: acc = 0.29134, nmi = 0.52612, ari = 0.14461  ; loss= 7.11497
Iter 840: acc = 0.29134, nmi = 0.52285, ari = 0.14414  ; loss= 6.56727
Iter 980: acc = 0.29134, nmi = 0.52282, ari = 0.14444  ; loss= 6.23553
Iter 1120: acc = 0.29057, nmi = 0.52283, ari = 0.14461  ; loss= 7.44327
Iter 1260: acc = 0.29212, nmi = 0.52328, ari = 0.14568  ; loss= 6.62974
Iter 1400: acc = 0.29212, nmi = 0.52262, ari = 0.14502  ; loss= 5.9629
Iter 1540: acc = 0.29212, nmi = 0.52307, ari = 0.14430  ; loss= 6.27522
Iter 1680: acc = 0.28825, nmi = 0.52183, ari = 0.14330  ; loss= 6.64078
Iter 1820: acc = 0.29134, nmi = 0.52427, ari = 0.14427  ; loss= 7.33977
Iter 1960: ac

# Advanced (Train AE and Clustering Simultaneously)

**Important:** Do not forget to reinitialize the AE before running the code below

In [90]:
from tensorflow.keras.optimizers import SGD

N_CLUSTERS = 72

# (re)initialize clustering layer
clustering_layer = ClusteringLayer(N_CLUSTERS, name='clustering')(encoder.output)

# Train AE and clustering layer at the same time
model = keras.Model(inputs=inputs,
            outputs=[clustering_layer, sequence_autoencoder.output])
pretrain_optimizer = SGD(learning_rate=0.01, momentum=0.9)

model.compile(loss=['kld', 'mse'], loss_weights=[0.1, 1], optimizer='adam')

In [91]:
for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q, _  = model.predict(X, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y is not None:
            acc = np.round(attack.cluster_acc(y, y_pred), 5)
            nmi = np.round(metrics.normalized_mutual_info_score(y, y_pred), 5)
            ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f' % (ite, acc, nmi, ari), ' ; loss=', loss)

        # check stop criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, X.shape[0])]
    loss = model.train_on_batch(x=tf.gather(X, indices=idx), y=[p[idx], tf.gather(X, indices=idx)])
    index = index + 1 if (index + 1) * batch_size <= X.shape[0] else 0

Iter 0: acc = 0.10046, nmi = 0.14096, ari = 0.02553  ; loss= [14.77079  0.49049  0.02395]
Iter 140: acc = 0.08578, nmi = 0.10227, ari = 0.00949  ; loss= [1.503691e+01 7.000000e-05 2.427000e-02]
Iter 280: acc = 0.10046, nmi = 0.15124, ari = 0.02319  ; loss= [14.88898  0.       0.02138]
Iter 420: acc = 0.10896, nmi = 0.17563, ari = 0.03253  ; loss= [1.484727e+01 1.000000e-05 2.219000e-02]
Iter 560: acc = 0.13833, nmi = 0.23023, ari = 0.04790  ; loss= [1.498289e+01 1.000000e-05 1.916000e-02]
Iter 700: acc = 0.19474, nmi = 0.32634, ari = 0.07177  ; loss= [1.481378e+01 3.000000e-05 1.844000e-02]
Iter 840: acc = 0.20711, nmi = 0.37116, ari = 0.08635  ; loss= [1.501879e+01 5.000000e-05 1.900000e-02]
Iter 980: acc = 0.20015, nmi = 0.36085, ari = 0.07436  ; loss= [1.491256e+01 1.800000e-04 1.703000e-02]
Iter 1120: acc = 0.20170, nmi = 0.37403, ari = 0.06870  ; loss= [1.487374e+01 4.000000e-04 1.658000e-02]
Iter 1260: acc = 0.21870, nmi = 0.40721, ari = 0.08320  ; loss= [1.483837e+01 2.600000e-0